In [1]:
import os
os.environ['SPARK_CLASSPATH'] = "/home/ubuntu/postgresql-9.3-1103-jdbc41.jar"

In [2]:
import findspark

In [3]:
findspark.init()

In [10]:
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row
from pyspark.sql import DataFrameReader

In [5]:
sc = SparkContext(appName = "BuildProductRecommendations_event")

In [6]:
sqlContext = SQLContext(sc)

In [7]:
properties = {
    "user": "postgres_user",
    "password": "admin"
}

In [8]:
url="jdbc:postgresql://client.tuple-mia.com:5432/postgres_db"

In [12]:
df = DataFrameReader(sqlContext).jdbc(url = url, table = 'trans', properties = properties)

In [9]:
df = sqlContext.read.format('jdbc').\
    options(url = url, dbtable = 'tablename').\
    
    load()

Py4JJavaError: An error occurred while calling o25.load.
: org.postgresql.util.PSQLException: The server requested password-based authentication, but no password was provided.
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:461)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:188)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:64)
	at org.postgresql.jdbc2.AbstractJdbc2Connection.<init>(AbstractJdbc2Connection.java:143)
	at org.postgresql.jdbc3.AbstractJdbc3Connection.<init>(AbstractJdbc3Connection.java:29)
	at org.postgresql.jdbc3g.AbstractJdbc3gConnection.<init>(AbstractJdbc3gConnection.java:21)
	at org.postgresql.jdbc4.AbstractJdbc4Connection.<init>(AbstractJdbc4Connection.java:38)
	at org.postgresql.jdbc4.Jdbc4Connection.<init>(Jdbc4Connection.java:24)
	at org.postgresql.Driver.makeConnection(Driver.java:412)
	at org.postgresql.Driver.connect(Driver.java:280)


In [13]:
df.take(5)

[Row(product_id=u'com.gopaktor.subscription.v1.premium.1m', receipt_id=u'1984955', revenue=31.28, purchase_date=datetime.datetime(2016, 9, 7, 13, 59, 45), renewal=0, userid=u'3893506', insider=0, platform=u'ios', quantity=0),
 Row(product_id=u'com.gopaktor.subscription.v1.premium.1m', receipt_id=u'2170195', revenue=31.28, purchase_date=datetime.datetime(2016, 9, 25, 3, 6, 11), renewal=0, userid=u'3793254', insider=0, platform=u'ios', quantity=0),
 Row(product_id=u'com.gopaktor.subscription.v1.premium.1m', receipt_id=u'2259995', revenue=31.28, purchase_date=datetime.datetime(2016, 10, 1, 2, 27, 37), renewal=0, userid=u'6148794', insider=0, platform=u'ios', quantity=0),
 Row(product_id=u'com.gopaktor.subscription.v4.premium.1m', receipt_id=u'1980761', revenue=31.28, purchase_date=datetime.datetime(2016, 10, 1, 6, 39, 16), renewal=0, userid=u'6149220', insider=0, platform=u'android', quantity=0),
 Row(product_id=u'com.gopaktor.subscription.v4.premium.1m', receipt_id=u'1980865', revenue=31

In [ ]:
df.count()

In [ ]:
type(df)

In [ ]:
df2 = df.select('from_id', 'to_id', 'actiontype')

In [ ]:
df2.take(5)

In [ ]:
data = df2.collect()

In [ ]:
import pandas as pd

In [ ]:
data_df = pd.DataFrame(data, columns=df2.columns)

In [ ]:
data_df.head()

In [ ]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml', 'r') as f:
    doc = yaml.load(f)

In [ ]:
doc['column_map']['EVENT_LOG']

In [ ]:
data_df.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace = True)
data_df.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace = True)

In [ ]:
data_df.head()

In [ ]:
from pyspark.mllib.recommendation import ALS

In [ ]:
n_prod_views_rdd = sqlContext.createDataFrame(data_df).rdd

In [ ]:
n_prod_views_rdd.take(5)

In [ ]:
training_rdd, test_rdd = n_prod_views_rdd.randomSplit([8,2], 1345)

In [ ]:
training_rdd.distinct().count()

In [ ]:
test_for_predict_rdd = test_rdd.map(lambda x : (x[0], x[1]))

In [ ]:
print('Training RDD\n', training_rdd.take(5))
print('\nTest for Prediction RDD\n', test_for_predict_rdd.take(5))

In [ ]:
import math

In [ ]:
seed = 49247
iterations = 10
lambdas = [0.01, 0.1]
ranks = [16]
alphas = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 40.0, 80.0]
errors = [0 for x in range(len(alphas) * len(ranks) * len(lambdas))]
err_index = 0

In [ ]:
for lambda_ in  lambdas:
    for rank in ranks:
        for alpha in alphas:
            model = ALS.trainImplicit(training_rdd, rank, seed=seed, iterations=iterations,
                                           lambda_ = lambda_, alpha = alpha)
            predictions = model.predictAll(test_for_predict_rdd).map(lambda r : ((r[0], r[1]), r[2]))
            views_and_preds = test_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
            error = math.sqrt(views_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
            errors[err_index] = error
            err_index += 1
            print('For rank {0} at alpha: {1} and lambda: {2}, the RMSE is{3}'.format(rank, alpha, lambda_, error))

In [ ]:
model.recommendProducts(6,10)

In [ ]:
all_user = model.recommendProductsForUsers(20)

In [ ]:
all_user.count()

In [ ]:
all_user.take(1)

In [ ]:
rec_all = all_user.collect()

In [ ]:
len(rec_all)

In [ ]:
rec_all[0]

In [ ]:
rec = []

In [ ]:
for x in range(len(rec_all)):
    for y in range(17):
            v = rec_all[x][1][y][0:3]
            d = list(v)
            rec.append(d)

In [ ]:
rec[2]

In [ ]:
rec_df = pd.DataFrame(rec, columns=['cust_id', 'rec_product', 'rating'])

In [ ]:
rec_df.head()

In [ ]:
rec_df.shape

In [ ]:
model1 = ALS.trainImplicit(n_prod_views_rdd, rank=16, seed=seed, iterations=iterations,alpha=0.01)

In [ ]:
my_rec = model1.recommendProductsForUsers(20).collect()

In [ ]:
len(my_rec)

In [ ]:
rec2 = []

In [ ]:
for x in range(len(rec_all)):
    for y in range(17):
            v = rec_all[x][1][y][0:3]
            d = list(v)
            rec2.append(d)

In [ ]:
rec_df1 = pd.DataFrame(rec2, columns=['cust_id', 'rec_product', 'rating'])

In [ ]:
rec_df1.head()

In [ ]:
rec_df1.shape